In [1]:
import pandas as pd

df_stop_times = pd.read_csv("gtfs_data/stop_times.csv")
df_stops = pd.read_csv("gtfs_data/stops.csv")
df_translations = pd.read_csv("gtfs_data/translations.csv")

df_stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,87____:049::8728600:8889011:5:715:20241116,06:27:00,06:27:00,8728600,1,NaN,0,1,NaN
1,87____:049::8728600:8889011:5:715:20241116,06:40:00,06:40:00,8728671,2,NaN,1,1,NaN
2,87____:049::8728600:8889011:5:715:20241116,06:49:00,06:49:00,8728673,3,NaN,0,0,NaN
3,87____:049::8728600:8889011:5:715:20241116,07:01:00,07:01:00,8728654,4,NaN,0,0,NaN
4,87____:049::8728600:8889011:5:715:20241116,07:15:00,07:15:00,8889011,5,NaN,1,1,NaN


In [2]:
df_translations.head(10)

,table_name,field_name,language,translation,record_id,record_sub_id,field_value
0,stops,stop_name,nl,Ingelmunster,NaN,NaN,Ingelmunster
1,stops,stop_name,de,Ingelmunster,NaN,NaN,Ingelmunster
2,stops,stop_name,en,Ingelmunster,NaN,NaN,Ingelmunster
3,stops,stop_name,nl,Izegem,NaN,NaN,Izegem
4,stops,stop_name,de,Izegem,NaN,NaN,Izegem
5,stops,stop_name,en,Izegem,NaN,NaN,Izegem
6,stops,stop_name,nl,Roeselare,NaN,NaN,Roulers
7,trips,trip_headsign,nl,Roeselare,NaN,NaN,Roulers
8,stops,stop_name,de,Roeselare / Roulers,NaN,NaN,Roulers
9,trips,trip_headsign,de,Roeselare / Roulers,NaN,NaN,Roulers


In [3]:
df_stops.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,platform_code,stop_timezone
0,8015190,NaN,Herzogenrath,NaN,50.87090,6.094400,NaN,NaN,0,NaN,NaN,NaN
1,8015199,NaN,Aix-la-Chapelle Ouest,NaN,50.78078,6.070550,NaN,NaN,0,NaN,NaN,NaN
2,8015345,NaN,Aachen Hbf (DE),NaN,50.76776,6.091390,NaN,NaN,0,NaN,NaN,NaN
3,8015458,NaN,Köln Hbf (DE),NaN,50.94308,6.958970,NaN,NaN,0,NaN,NaN,NaN
4,8061676,NaN,Frankfurt Flugh (DE),NaN,50.13083,8.663611,NaN,NaN,0,NaN,NaN,NaN


In [4]:
import pandas as pd

# Filter the translations DataFrame to include only Dutch translations
df_translations_nl = df_translations[df_translations['language'] == 'nl']

# Merge the DataFrames on the French stop names
df_translated = df_stops.merge(df_translations_nl, left_on='stop_name', right_on='field_value', how='left')

# Replace the French stop names with Dutch translations
df_translated['stop_name'] = df_translated['translation']
df_translated = df_translated[['stop_id', 'stop_name']]

In [5]:
df_translations_nl.head()

,table_name,field_name,language,translation,record_id,record_sub_id,field_value
0,stops,stop_name,nl,Ingelmunster,NaN,NaN,Ingelmunster
3,stops,stop_name,nl,Izegem,NaN,NaN,Izegem
6,stops,stop_name,nl,Roeselare,NaN,NaN,Roulers
7,trips,trip_headsign,nl,Roeselare,NaN,NaN,Roulers
12,stops,stop_name,nl,Poperinge,NaN,NaN,Poperinge


In [6]:
df_stop_times["stop_id"] = df_stop_times["stop_id"].astype(str)

df_translated["stop_id"] = df_translated["stop_id"].astype(str)

In [7]:
df_merged = pd.merge(df_stop_times, df_translated, on='stop_id')

In [8]:
df_merged.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,stop_name
0,87____:049::8728600:8889011:5:715:20241116,06:27:00,06:27:00,8728600,1,NaN,0,1,NaN,NaN
1,87____:049::8728600:8889011:5:715:20241116,06:40:00,06:40:00,8728671,2,NaN,1,1,NaN,NaN
2,87____:049::8728600:8889011:5:715:20241116,06:49:00,06:49:00,8728673,3,NaN,0,0,NaN,NaN
3,87____:049::8728600:8889011:5:715:20241116,07:01:00,07:01:00,8728654,4,NaN,0,0,NaN,NaN
4,87____:049::8728600:8889011:5:715:20241116,07:15:00,07:15:00,8889011,5,NaN,1,1,NaN,Mouscron-Frontière


In [9]:
df_cleaned = df_merged[["trip_id", "departure_time", "stop_id", "stop_sequence", "stop_name"]]

In [10]:
# Split the departure_time column into hours and minutes
df_cleaned['hours'] = df_cleaned['departure_time'].str.split(':', expand=True)[0]
df_cleaned['minutes'] = df_cleaned['departure_time'].str.split(':', expand=True)[1]

# Convert hours and minutes to integers to remove leading zeros
df_cleaned['hours'] = df_cleaned['hours'].astype(int)
df_cleaned['minutes'] = df_cleaned['minutes'].astype(int)


C:\Users\Jelle\AppData\Local\Temp\ipykernel_43724\699540803.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['hours'] = df_cleaned['departure_time'].str.split(':', expand=True)[0]
C:\Users\Jelle\AppData\Local\Temp\ipykernel_43724\699540803.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['minutes'] = df_cleaned['departure_time'].str.split(':', expand=True)[1]
C:\Users\Jelle\AppData\Local\Temp\ipykernel_43724\699540803.py:6: SettingWithCopyWarning: 
A value is trying to be set on 

In [11]:
df_cleaned.head()

,trip_id,departure_time,stop_id,stop_sequence,stop_name,hours,minutes
0,87____:049::8728600:8889011:5:715:20241116,06:27:00,8728600,1,NaN,6,27
1,87____:049::8728600:8889011:5:715:20241116,06:40:00,8728671,2,NaN,6,40
2,87____:049::8728600:8889011:5:715:20241116,06:49:00,8728673,3,NaN,6,49
3,87____:049::8728600:8889011:5:715:20241116,07:01:00,8728654,4,NaN,7,1
4,87____:049::8728600:8889011:5:715:20241116,07:15:00,8889011,5,Mouscron-Frontière,7,15


In [12]:
mask = df_cleaned['stop_name'].isin(['Brussel-Noord'])
df_cleaned[mask].head()

,trip_id,departure_time,stop_id,stop_sequence,stop_name,hours,minutes
25296,88____:049::8814001:8811601:17:747:20241117,06:01:00,8812005,5,Brussel-Noord,6,1
25297,88____:049::8814001:8811601:17:747:20241117,06:01:00,8812005,5,Brussel-Noord,6,1
55168,88____:046::8821006:8814001:20:654:20241130,06:46:00,8812005,16,Brussel-Noord,6,46
55169,88____:046::8821006:8814001:20:654:20241130,06:46:00,8812005,16,Brussel-Noord,6,46
55245,88____:046::8814001:8821006:20:724:20250413,06:18:00,8812005,5,Brussel-Noord,6,18


In [13]:
# Step 1: Read the graph.lp file and extract the list of nodes
nodes = []
with open('graph_all_filtered.lp', 'r') as file:
    for line in file:
        if line.startswith('node('):
            node = line.split('(')[1].split(',')[0]
            nodes.append(node)
print(nodes)

['brussel_noord', 'brussel_noord', 'brussel_noord', 'brussel_noord', 'brussel_noord', 'brussel_noord', 'brussel_congres', 'brussel_centraal', 'brussel_zuid', 'brussel_zuid', 'brussel_zuid', 'brussel_zuid', 'brussel_zuid', 'la_louviere_centrum', "'s_gravenbrakel", "'s_gravenbrakel", 'ecaussinnes', 'luttre', 'luttre', 'bergen', 'bergen', 'zottegem', 'zottegem', 'geraardsbergen', 'vorst_oost', 'ukkel_kalevoet', 'linkebeek', 'sint_genesius_rode', 'waterloo', 'eigenbrakel', 'lillois', 'nijvel', 'marchienne_au_pont', 'charleroi_centraal', 'charleroi_centraal', 'charleroi_centraal', 'luik_guillemins', 'luik_guillemins', 'luik_guillemins', 'flemalle_haute', 'flemalle_haute', 'hoei', 'statte', 'andenne', 'namen', 'namen', 'namen', 'namen', 'seraing', 'kontich_lint', 'kontich_lint', 'kontich_lint', 'lier', 'auvelais', 'tamines', 'chatelet', 'leuven', 'leuven', 'leuven', 'leuven', 'leuven', 'heverlee', 'waver', 'ottignies', 'ottignies', 'ottignies', 'court_saint_etienne', 'gembloers', 'gembloers'

In [20]:
df_cleaned['stop_name'] = df_cleaned['stop_name'].str.lower().str.replace('-', '_')

df_filtered = df_cleaned[df_cleaned['stop_name'].isin(nodes)]
df_filtered.drop_duplicates()
# Converteer de DataFrame naar ASP-regels
asp_rules = []
for index, row in df_filtered.iterrows():
    if not 'nan' in str(row['stop_name']):
        if not f"departure_time({row['stop_id']}, '{row['hours']}', '{row['minutes']}')." in asp_rules:
            asp_rules.append(f"departure_time({row['stop_id']}, '{row['hours']}', '{row['minutes']}').")
        if not f"stop_id({row['stop_id']}, {row['stop_sequence']})." in asp_rules:
            asp_rules.append(f"stop_id({row['stop_id']}, {row['stop_sequence']}).")
        if not f"stop_sequence({row['stop_id']}, {row['stop_sequence']})." in asp_rules:    
            asp_rules.append(f"stop_sequence({row['stop_id']}, {row['stop_sequence']}).")
        if not f"stop_name({row['stop_id']}, '{row['stop_name']}')." in asp_rules:
            asp_rules.append(f"stop_name({row['stop_id']}, '{row['stop_name']}').")


asp_program = "\n".join(asp_rules)

# Print of sla het ASP-programma op
with open('schedule.lp', 'w') as file:
    file.write(asp_program)

C:\Users\Jelle\AppData\Local\Temp\ipykernel_43724\4241986791.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['stop_name'] = df_cleaned['stop_name'].str.lower().str.replace('-', '_')


In [18]:
df_cleaned

,trip_id,departure_time,stop_id,stop_sequence,stop_name,hours,minutes
0,87____:049::8728600:8889011:5:715:20241116,06:27:00,8728600,1,NaN,6,27
1,87____:049::8728600:8889011:5:715:20241116,06:40:00,8728671,2,NaN,6,40
2,87____:049::8728600:8889011:5:715:20241116,06:49:00,8728673,3,NaN,6,49
3,87____:049::8728600:8889011:5:715:20241116,07:01:00,8728654,4,NaN,7,1
4,87____:049::8728600:8889011:5:715:20241116,07:15:00,8889011,5,mouscron_frontière,7,15
...,...,...,...,...,...,...,...
902665,88____:046::8865003:8864345:6:2130:20251213,21:21:00,8864311,4,forrières,21,21
902666,88____:046::8865003:8864345:6:2130:20251213,21:25:00,8864006,5,rochefort_jemelle,21,25
902667,88____:046::8865003:8864345:6:2130:20251213,21:25:00,8864006,5,rochefort_jemelle,21,25
902668,88____:046::8865003:8864345:6:2130:20251213,21:30:00,8864345,6,marloie,21,30


In [16]:
df_filtered

,trip_id,departure_time,stop_id,stop_sequence,stop_name,hours,minutes
6,88____:049::8889011:8896008:3:748:20241116,07:18:00,8885704,6,moeskroen,7,18
7,88____:049::8889011:8896008:3:748:20241116,07:18:00,8885704,6,moeskroen,7,18
8,88____:049::8889011:8896008:3:748:20241116,07:48:00,8896008,7,kortrijk,7,48
9,88____:049::8889011:8896008:3:748:20241116,07:48:00,8896008,7,kortrijk,7,48
10,88____:049::8892007:8896008:6:2516:20241121,23:43:00,8892007,1,gent_sint_pieters,23,43
...,...,...,...,...,...,...,...
902662,88____:046::8865003:8864345:6:2130:20251213,20:56:00,8865003,1,libramont,20,56
902666,88____:046::8865003:8864345:6:2130:20251213,21:25:00,8864006,5,rochefort_jemelle,21,25
902667,88____:046::8865003:8864345:6:2130:20251213,21:25:00,8864006,5,rochefort_jemelle,21,25
902668,88____:046::8865003:8864345:6:2130:20251213,21:30:00,8864345,6,marloie,21,30


In [21]:
asp_rules

["departure_time(8885704, '7', '18').",
 'stop_id(8885704, 6).',
 'stop_sequence(8885704, 6).',
 "stop_name(8885704, 'moeskroen').",
 "departure_time(8896008, '7', '48').",
 'stop_id(8896008, 7).',
 'stop_sequence(8896008, 7).',
 "stop_name(8896008, 'kortrijk').",
 "departure_time(8892007, '23', '43').",
 'stop_id(8892007, 1).',
 'stop_sequence(8892007, 1).',
 "stop_name(8892007, 'gent_sint_pieters').",
 "departure_time(8896008, '25', '16').",
 'stop_id(8896008, 6).',
 'stop_sequence(8896008, 6).',
 "departure_time(8895208, '5', '51').",
 'stop_id(8895208, 1).',
 'stop_sequence(8895208, 1).',
 "stop_name(8895208, 'zottegem').",
 "departure_time(8895455, '6', '20').",
 'stop_id(8895455, 5).',
 'stop_sequence(8895455, 5).',
 "stop_name(8895455, 'burst').",
 "departure_time(8895877, '6', '30').",
 'stop_id(8895877, 6).',
 'stop_sequence(8895877, 6).',
 "stop_name(8895877, 'ede').",
 "departure_time(8895869, '6', '36').",
 'stop_id(8895869, 7).',
 'stop_sequence(8895869, 7).',
 "stop_name(